<a href="https://colab.research.google.com/github/anyaiyer/melanoma-detection/blob/main/melanoma_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Interactive Application of Melanoma Model

Scroll down to use the finished application of this project. Code for building the widgets comes from [Fastai fastbook, chapter 2](https://github.com/fastai/fastbook/blob/master/02_production.ipynb). 

In [1]:
#@title
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
#@title
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [3]:
#@title
from fastai import *

In [4]:
#@title
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
#@title
root_dir = "/content/gdrive/My Drive/Colab Notebooks/cancer_detection/"

In [6]:
#@title
import pandas as pd

In [7]:
#@title
label_info = pd.read_csv(f"{root_dir}/train.csv")

In [8]:
#@title
def label_func(fname): # function grabs the label from the image name
    label =  label_info.loc[label_info['image_name'] == fname.name[:-4]]['target'].item()
    return label

In [9]:
#@title
dblock = DataBlock(blocks    = (ImageBlock, CategoryBlock), # independent & dependent variables
                   get_items = get_image_files, # gets the images
                   get_y     = label_func, # gets the labels
                   splitter  = RandomSplitter(valid_pct=0.1,seed=42), # 20% used for the validation set
                   batch_tfms=aug_transforms(mult=1.0, # data augmentation
                                             do_flip=True, 
                                             flip_vert=True,
                                             max_rotate=10.0,
                                             min_zoom=1.0,
                                             max_zoom=1.1,
                                             max_lighting=0.2, 
                                             max_warp=0.0,
                                             p_affine=0.0,
                                             p_lighting=0.0,
                                             xtra_tfms=None,
                                             size=None, 
                                             mode='bilinear',
                                             pad_mode='reflection',
                                             align_corners=True,
                                             batch=False, min_scale=1.0), 
                   item_tfms = Resize(224)) # resizing the image

In [10]:
#@title
path = Path("/content/gdrive/My Drive/Colab Notebooks/cancer_detection/train6") # file path for the image

In [11]:
#@title
dls = dblock.dataloaders(path)  
dls.bs = 32 # batch size

In [15]:
#@title
predictor = Path('/content/gdrive/My Drive/Colab Notebooks/cancer_detection/model_1.pth') # path of model

In [16]:
#@title
predictor = cnn_learner(dls, resnet50, metrics=metrics)
predictor.load(f"{root_dir}/model_1") # load model

In [17]:
#@title
from ipywidgets import Button, HBox, VBox
import ipywidgets as widgets
from IPython.display import display

In [18]:
#@title
btn_upload = widgets.FileUpload(layout=widgets.Layout(width='%10', height='40px'),
    style= {'button_color': 'lightblue'})

In [19]:
#@title
btn_upload = SimpleNamespace(data = ['/content/gdrive/My Drive/Colab Notebooks/cancer_detection/test1/ISIC_3609081.jpg'])

In [20]:
#@title
out_pl = widgets.Output()

In [22]:
#@title
btn_run = widgets.Button(description='Classify', layout=widgets.Layout(width='%10', height='40px'),style= {'button_color': 'lightgreen'})

In [23]:
#@title
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = predictor.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [24]:
#@title
btn_upload = widgets.FileUpload(layout=widgets.Layout(width='%10', height='40px'),
    style= {'button_color': 'lightblue'})

In [25]:
#@title
display_pred = VBox([widgets.Label('Select your image'), 
      btn_upload, btn_run, out_pl, lbl_pred])
display_pred